In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import eda

## 유동인구

**성연령**

In [2]:
age = eda.flow_concat('AGE','../data/01_SKT/*.csv')
age = eda.flow_preprocessing(age)
age = eda.flow_melting(age)
age['AGE_CD'] = age['VAR'].apply(lambda x: x[-3:] if x[-4] == '_' else x[-4:])
age['SEX_CD'] = age['VAR'].str[0].map({'M':'M', 'W':'F'})
age = age[['STD_YMD', 'GU_NM', 'HDONG_NM', 'SEX_CD', 'AGE_CD', 'POP_CNT']]
age_dict = {x: int(x[:2])for x in sorted(age['AGE_CD'].unique())}
age['AGE_CD'] = age['AGE_CD'].map(age_dict)
age['AGE_CD'] = age['AGE_CD'].apply(lambda x: 20 if x < 20 else x)
age['AGE_CD'] = age['AGE_CD'].apply(lambda x: 65 if x > 65 else x)
age = age.groupby(['STD_YMD', 'GU_NM', 'HDONG_NM'])['POP_CNT'].sum().reset_index()
age_per = pd.read_csv('../data/08_Percent_Matrix/age_per.csv')
age = pd.concat([age, age_per],axis=1)


print(age.shape)
age.head()

(16629, 14)


,STD_YMD,GU_NM,HDONG_NM,POP_CNT,AG_20_F,AG_30_F,AG_40_F,AG_50_F,AG_60_F,AG_20_M,AG_30_M,AG_40_M,AG_50_M,AG_60_M
0,2019-02-01,대구 수성구,고산1동,133361.08,0.094684,0.066612,0.117062,0.099566,0.070804,0.104180,0.085641,0.131000,0.129117,0.101335
1,2019-02-01,대구 수성구,고산2동,254912.57,0.077246,0.067818,0.099493,0.087825,0.068341,0.096442,0.106579,0.147573,0.143406,0.105276
2,2019-02-01,대구 수성구,고산3동,57764.08,0.091888,0.073853,0.112215,0.098319,0.071726,0.098732,0.089873,0.129602,0.128439,0.105352
3,2019-02-01,대구 수성구,두산동,125647.42,0.077910,0.078633,0.113528,0.091201,0.078017,0.082605,0.106474,0.141090,0.126571,0.103969
4,2019-02-01,대구 수성구,만촌1동,132277.61,0.079484,0.067034,0.095607,0.091221,0.075020,0.098307,0.100490,0.136893,0.139035,0.116909


**시간**

In [3]:
time = eda.flow_concat('TIME','../data/01_SKT/*.csv')
time = eda.flow_preprocessing(time)

print(time.shape)
time.head()

(16629, 27)


,STD_YMD,GU_NM,HDONG_NM,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,TMST_06,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
0,2019-02-01,서울 중구,소공동,1496.35,1022.83,706.51,547.77,1673.02,3561.05,9195.82,...,12355.91,13096.69,9317.64,15029.74,14135.83,9074.02,6032.26,6047.05,4806.65,2956.11
1,2019-02-01,서울 중구,회현동,1692.37,1117.26,839.57,653.41,1920.52,4687.95,10936.47,...,18210.30,20292.65,12527.29,21454.63,18214.50,11914.26,9800.64,9341.24,6884.91,4034.56
2,2019-02-01,서울 중구,명동,2226.91,1726.10,1145.43,894.97,2513.32,5268.51,13857.93,...,20332.71,21458.64,14434.56,25855.14,23107.23,13460.05,9797.40,8719.30,7203.45,4845.48
3,2019-02-01,서울 중구,필동,698.73,440.70,385.99,275.64,497.41,1990.42,4679.79,...,6900.66,9789.54,6205.74,13277.25,12101.51,5698.58,4464.46,4088.53,3385.14,1746.28
4,2019-02-01,서울 중구,장충동,768.34,447.23,429.90,349.46,507.74,1310.67,2956.49,...,5187.02,5211.60,3656.54,6841.98,5810.71,4463.99,2807.25,2626.12,2415.41,1398.94


In [4]:
total = time.loc[: , 'TMST_00':].sum(axis=1)
for i in time.columns[3:].str[-2:]:
    time[f'TMST_{i}'] = time[f'TMST_{i}'] / total

In [5]:
flow = pd.merge(age, time, on=['STD_YMD', 'GU_NM', 'HDONG_NM'])

## 카드매출

**내국인**

In [6]:
resid = pd.read_csv('../data/02_Shinhan_Card/shinhan_dataset(내국인).txt', sep='\t')
resid = eda.card_preprocessing(resid)
resid.rename(columns={'STD_DD':'STD_YMD', 'DONG_NM':'HDONG_NM'}, inplace=True)
sales = resid.groupby(['STD_YMD', 'GU_NM', 'HDONG_NM'])['USE_AMT'].sum().reset_index()

renaming = ['LODGE', 'LEIS_ITEM', 'LEIS', 'ELEC', 'KITCH', 'FUEL', 'OPTICAL', 'DIST',
            'OFFICE', 'CAR_SERVICE', 'MEDIAN', 'HYGIENE', 'REST', 'GROCERY', 'REPAIR',
            'HOBBY', 'FURN', 'H_APPL', 'CLOTHES', 'ACC', 'BOOK', 'CROP', 'CAR_SALES']
df_list = []
for name in list(map(lambda x: x.lower(), renaming)):
    df_list.append(pd.read_csv(f'../data/08_Percent_Matrix/{name}_per.csv'))
    
sales = pd.concat([sales]+df_list, axis=1)

print(sales.shape)
sales.head()

(16629, 234)


,STD_YMD,GU_NM,HDONG_NM,USE_AMT,SH_LODGE_20_F,SH_LODGE_30_F,SH_LODGE_40_F,SH_LODGE_50_F,SH_LODGE_60_F,SH_LODGE_20_M,...,SH_CAR_SALES_20_F,SH_CAR_SALES_30_F,SH_CAR_SALES_40_F,SH_CAR_SALES_50_F,SH_CAR_SALES_60_F,SH_CAR_SALES_20_M,SH_CAR_SALES_30_M,SH_CAR_SALES_40_M,SH_CAR_SALES_50_M,SH_CAR_SALES_60_M
0,2019-02-01,대구 수성구,고산1동,411993,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,2019-02-01,대구 수성구,고산2동,386930,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,2019-02-01,대구 수성구,고산3동,336754,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,2019-02-01,대구 수성구,두산동,681681,0.150946,0.042812,0.095291,0.057036,0.00000,0.097224,...,0.178383,0.0,0.0,0.0,0.007519,0.0,0.0,0.469173,0.0,0.344925
4,2019-02-01,대구 수성구,만촌1동,540787,0.041777,0.022063,0.177765,0.105387,0.07255,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [7]:
table = pd.merge(flow, sales, on=['STD_YMD', 'GU_NM', 'HDONG_NM'])
table.head()

,STD_YMD,GU_NM,HDONG_NM,POP_CNT,AG_20_F,AG_30_F,AG_40_F,AG_50_F,AG_60_F,AG_20_M,...,SH_CAR_SALES_20_F,SH_CAR_SALES_30_F,SH_CAR_SALES_40_F,SH_CAR_SALES_50_F,SH_CAR_SALES_60_F,SH_CAR_SALES_20_M,SH_CAR_SALES_30_M,SH_CAR_SALES_40_M,SH_CAR_SALES_50_M,SH_CAR_SALES_60_M
0,2019-02-01,대구 수성구,고산1동,133361.08,0.094684,0.066612,0.117062,0.099566,0.070804,0.104180,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,2019-02-01,대구 수성구,고산2동,254912.57,0.077246,0.067818,0.099493,0.087825,0.068341,0.096442,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,2019-02-01,대구 수성구,고산3동,57764.08,0.091888,0.073853,0.112215,0.098319,0.071726,0.098732,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,2019-02-01,대구 수성구,두산동,125647.42,0.077910,0.078633,0.113528,0.091201,0.078017,0.082605,...,0.178383,0.0,0.0,0.0,0.007519,0.0,0.0,0.469173,0.0,0.344925
4,2019-02-01,대구 수성구,만촌1동,132277.61,0.079484,0.067034,0.095607,0.091221,0.075020,0.098307,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [8]:
table['SH_OPTICAL_20_M']

,SH_OPTICAL_20_M,SH_OPTICAL_20_M
0,0.000000,0.000000
1,0.812410,0.812410
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
16624,0.000000,0.000000
16625,0.414141,0.414141
16626,0.084381,0.084381
16627,0.000000,0.000000


In [9]:
table.columns.value_counts()

SH_OFFICE_30_M    2
SH_OFFICE_50_F    2
SH_DIST_40_F      2
SH_OFFICE_20_M    2
SH_OFFICE_60_F    2
                 ..
SH_LEIS_60_F      1
SH_REST_40_F      1
SH_CROP_50_F      1
SH_KITCH_40_M     1
SH_FUEL_30_M      1
Length: 239, dtype: int64

## 유통

In [10]:
logistics = pd.read_excel('../data/04_GS_Retail/gs_dataset.xlsx',
                          sheet_name='TB_DD_STR_PURCH_CLS',
                          parse_dates=['OPER_DT'])
logistics['BOR'] = logistics['PVN'].str[:2] + ' ' + logistics['BOR']
logistics.rename(columns={'OPER_DT':'STD_YMD', 'BOR':'GU_NM', 'ADMD':'HDONG_NM'},
                 inplace=True)
logistics.drop('PVN', axis=1, inplace=True)
print(logistics.shape)
logistics.head()

(14460, 12)


,STD_YMD,GU_NM,HDONG_NM,AMT_IND,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P
0,2019-02-01,대구 수성구,고산1동,1.064336,0.450848,0.172221,0.295458,0.023263,0.044679,0.005075,0.004820,0.003637
1,2019-02-01,대구 수성구,고산2동,1.076418,0.457937,0.196274,0.295564,0.005521,0.034056,0.000844,0.008893,0.000913
2,2019-02-01,대구 수성구,고산3동,0.856071,0.359978,0.196602,0.376762,0.013408,0.029606,0.009117,0.008831,0.005697
3,2019-02-01,대구 수성구,두산동,0.724868,0.281969,0.175207,0.363202,0.024235,0.134191,0.000000,0.021196,0.000000
4,2019-02-01,대구 수성구,만촌1동,1.155728,0.400983,0.200454,0.310531,0.038868,0.035427,0.005284,0.005734,0.002719


In [11]:
logistics.columns = ['STD_YMD', 'GU_NM', 'HDONG_NM', 'AMT_IND', 'GS_MEAL',
                     'GS_SNACK', 'GS_DRINK', 'GS_HOME', 'GS_HEALTH', 'GS_HOBBY', 
                     'GS_SOCIAL', 'GS_PREG']

In [12]:
table = pd.merge(table, logistics, on=['STD_YMD', 'GU_NM', 'HDONG_NM'], how='left') \
          .fillna(0)

In [13]:
table.head()

,STD_YMD,GU_NM,HDONG_NM,POP_CNT,AG_20_F,AG_30_F,AG_40_F,AG_50_F,AG_60_F,AG_20_M,...,SH_CAR_SALES_60_M,AMT_IND,GS_MEAL,GS_SNACK,GS_DRINK,GS_HOME,GS_HEALTH,GS_HOBBY,GS_SOCIAL,GS_PREG
0,2019-02-01,대구 수성구,고산1동,133361.08,0.094684,0.066612,0.117062,0.099566,0.070804,0.104180,...,0.000000,1.064336,0.450848,0.172221,0.295458,0.023263,0.044679,0.005075,0.004820,0.003637
1,2019-02-01,대구 수성구,고산2동,254912.57,0.077246,0.067818,0.099493,0.087825,0.068341,0.096442,...,0.000000,1.076418,0.457937,0.196274,0.295564,0.005521,0.034056,0.000844,0.008893,0.000913
2,2019-02-01,대구 수성구,고산3동,57764.08,0.091888,0.073853,0.112215,0.098319,0.071726,0.098732,...,0.000000,0.856071,0.359978,0.196602,0.376762,0.013408,0.029606,0.009117,0.008831,0.005697
3,2019-02-01,대구 수성구,두산동,125647.42,0.077910,0.078633,0.113528,0.091201,0.078017,0.082605,...,0.344925,0.724868,0.281969,0.175207,0.363202,0.024235,0.134191,0.000000,0.021196,0.000000
4,2019-02-01,대구 수성구,만촌1동,132277.61,0.079484,0.067034,0.095607,0.091221,0.075020,0.098307,...,0.000000,1.155728,0.400983,0.200454,0.310531,0.038868,0.035427,0.005284,0.005734,0.002719


## 물류

In [14]:
delivery = pd.read_excel('../data/05_CJ_Olive_Networks/cj_dataset.xlsx')
delivery = eda.delivery_preprocessing(delivery)
delivery_group = delivery.groupby(['STD_YMD', 'GU_NM', 'HDONG_NM'])['INV_CNT'].sum().reset_index()
sharing_index = pd.read_csv('../data/sharing_index.csv', parse_dates=['STD_YMD'])
delivery_group = pd.merge(sharing_index, delivery_group, how='left').fillna(0)
delivery_group['INV_CNT'] = delivery_group['INV_CNT'].astype('int')
delivery_per = pd.read_csv('../data/08_Percent_Matrix/delivery_per.csv')
delivery = pd.concat([delivery_group, delivery_per], axis=1)

print(delivery.shape)
delivery.head()

(16629, 14)


,STD_YMD,GU_NM,HDONG_NM,INV_CNT,CJ_FURN,CJ_BOOK,CJ_H_APPL,CJ_HEALTH,CJ_SPORT,CJ_FOOD,CJ_PREG,CJ_CLOTH,CJ_ACC,CJ_BEAUTY
0,2019-02-01,대구 수성구,고산1동,333,0.006006,0.342342,0.042042,0.093093,0.012012,0.108108,0.003003,0.195195,0.045045,0.153153
1,2019-02-01,대구 수성구,고산2동,220,0.018182,0.331818,0.081818,0.081818,0.004545,0.059091,0.000000,0.168182,0.100000,0.154545
2,2019-02-01,대구 수성구,고산3동,348,0.040230,0.264368,0.060345,0.137931,0.008621,0.089080,0.005747,0.178161,0.074713,0.140805
3,2019-02-01,대구 수성구,두산동,137,0.007299,0.328467,0.051095,0.116788,0.000000,0.043796,0.000000,0.211679,0.102190,0.138686
4,2019-02-01,대구 수성구,만촌1동,186,0.010753,0.301075,0.064516,0.112903,0.000000,0.075269,0.005376,0.252688,0.064516,0.112903


In [15]:
table = pd.merge(table, delivery, on=['STD_YMD', 'GU_NM', 'HDONG_NM'], how='left')

## 질병

**서울**

In [16]:
seoul = pd.read_csv('../data/06_Epidemic_Datasets/Seoul_Covid.csv')
seoul = eda.seoul_preprocessing(seoul)
seoul['S_PT_CNT'] = 1
seoul = seoul[['S_PT_CNT']]
seoul = seoul.groupby('STD_YMD')['S_PT_CNT'].sum().reset_index()
period = pd.date_range('2020-01-24', '2020-05-31', freq='d')
period = pd.DataFrame(period, columns=['STD_YMD'])
seoul = pd.merge(period, seoul, how='left').fillna(0)
seoul['S_PT_CNT'] = seoul['S_PT_CNT'].astype('int')
seoul['CUM_CNT'] = seoul['S_PT_CNT'].cumsum()
seoul = seoul.query('STD_YMD >= "2020-02-01"')
seoul = seoul[['STD_YMD', 'S_PT_CNT']]

print(seoul.shape)
seoul.head()

AssertionError: <class 'numpy.ndarray'>

In [ ]:
daegu = pd.read_csv('../data/06_Epidemic_Datasets/Daegu_Covid.csv', 
                    encoding='cp949', parse_dates=['날짜'])
daegu.rename(columns={'날짜':'STD_YMD', '당일 확진자 수':'D_PT_CNT'}, inplace=True)
daegu = daegu[['STD_YMD', 'D_PT_CNT']]
daegu = pd.merge(period, daegu, how='left').fillna(0)
daegu['D_PT_CNT'] = daegu['D_PT_CNT'].astype('int')
daegu['CUM_CNT'] = daegu['D_PT_CNT'].cumsum()
daegu = daegu.query('STD_YMD >= "2020-02-01"')
daegu = daegu[['STD_YMD', 'D_PT_CNT']]

print(daegu.shape)
daegu.head()

In [ ]:
table = pd.merge(table, seoul, how='left').merge(daegu, how='left').fillna(0)
table['S_PT_CNT'] = table['S_PT_CNT'].astype('int')
table['D_PT_CNT'] = table['D_PT_CNT'].astype('int')

## 날씨

In [ ]:
df_list = [pd.read_csv('../data/07_External_Datasets/weather_{}.csv' \
                       .format(i), encoding='cp949') for i in [2019, 2020]]
weather = pd.concat(df_list)
weather = weather.fillna(0)
weather['RAIN'] = weather['일강수량(mm)'].apply(lambda x: 1 if x > 0 else 0)
weather = weather[['지점명', '일시', '평균기온(°C)', 'RAIN']]
weather.rename(columns={'지점명':'CT_NM', '일시':'STD_YMD',
                        '평균기온(°C)':'AV_TEMP'}, inplace=True)
weather['STD_YMD'] = weather['STD_YMD'].astype('datetime64')
print(weather.shape)
weather.head()

In [ ]:
table = pd.merge(table.assign(CT_NM=table['GU_NM'].str[:2]), weather, on=['STD_YMD', 'CT_NM'])

In [ ]:
table = pd.concat([table.drop(['POP_CNT', 'USE_AMT', 'INV_CNT', 'CT_NM'], axis=1),
                   table[['POP_CNT', 'INV_CNT', 'USE_AMT']]], axis=1)

In [ ]:
table.to_csv('../Data/table.csv', index=False)

In [ ]:
table

=============================================================

<pre>for 업종(renaming)
    for 성별(F,M)
        for 나이대(20, 30, 40, 50, 60)</pre>
        열 이름 : PER_SH_업종_나이_성별

In [ ]:
table['SH_LODGE_20_F']

In [ ]:
table['AMT_SH_LODGE_20_F'] = table['SH_LODGE_20_F']*table['USE_AMT']

In [ ]:
table['AMT_SH_LODGE_20_F']

In [ ]:
for y in ['F', 'M']:
    for z in ['20', '30', '40', '50', '60']:
        table['AMT_SH_LODGE_'+z+'_'+y] = table['SH_LODGE_'+z+'_'+y]*table['USE_AMT']

In [ ]:
table.head()

In [ ]:
renaming

In [ ]:
for x in renaming:
    for y in ['F', 'M']:
        for z in ['20', '30', '40', '50', '60']:
            table['AMT_SH_' + str(x) +'_'+ str(z) +'_'+ str(y)] = table['SH_'+str(x) +'_'+ str(z) +'_'+ str(y)] * table['USE_AMT']

In [ ]:
table.head()

In [ ]:
for x in renaming:
    for y in ['F', 'M']:
        for z in ['20', '30', '40', '50', '60']:
            table['AMT_SH_'+ x +'_'+ z +'_'+ y] = table['SH_' + x + '_' + z + '_' + y] * table['USE_AMT']

==================

In [20]:
table

,STD_YMD,GU_NM,HDONG_NM,POP_CNT,AG_20_F,AG_30_F,AG_40_F,AG_50_F,AG_60_F,AG_20_M,...,CJ_FURN,CJ_BOOK,CJ_H_APPL,CJ_HEALTH,CJ_SPORT,CJ_FOOD,CJ_PREG,CJ_CLOTH,CJ_ACC,CJ_BEAUTY
0,2019-02-01,대구 수성구,고산1동,133361.08,0.094684,0.066612,0.117062,0.099566,0.070804,0.104180,...,0.006006,0.342342,0.042042,0.093093,0.012012,0.108108,0.003003,0.195195,0.045045,0.153153
1,2019-02-01,대구 수성구,고산2동,254912.57,0.077246,0.067818,0.099493,0.087825,0.068341,0.096442,...,0.018182,0.331818,0.081818,0.081818,0.004545,0.059091,0.000000,0.168182,0.100000,0.154545
2,2019-02-01,대구 수성구,고산3동,57764.08,0.091888,0.073853,0.112215,0.098319,0.071726,0.098732,...,0.040230,0.264368,0.060345,0.137931,0.008621,0.089080,0.005747,0.178161,0.074713,0.140805
3,2019-02-01,대구 수성구,두산동,125647.42,0.077910,0.078633,0.113528,0.091201,0.078017,0.082605,...,0.007299,0.328467,0.051095,0.116788,0.000000,0.043796,0.000000,0.211679,0.102190,0.138686
4,2019-02-01,대구 수성구,만촌1동,132277.61,0.079484,0.067034,0.095607,0.091221,0.075020,0.098307,...,0.010753,0.301075,0.064516,0.112903,0.000000,0.075269,0.005376,0.252688,0.064516,0.112903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16624,2020-05-31,서울 중구,중림동,21982.16,0.118756,0.091626,0.068278,0.068286,0.065473,0.111816,...,0.013333,0.273333,0.026667,0.153333,0.000000,0.180000,0.000000,0.140000,0.093333,0.120000
16625,2020-05-31,서울 중구,청구동,15985.86,0.114121,0.079789,0.077776,0.070578,0.093974,0.116754,...,0.009434,0.245283,0.028302,0.103774,0.000000,0.264151,0.000000,0.188679,0.084906,0.075472
16626,2020-05-31,서울 중구,필동,53756.05,0.136558,0.079797,0.065272,0.072916,0.096838,0.119957,...,0.017241,0.275862,0.051724,0.051724,0.000000,0.120690,0.000000,0.206897,0.137931,0.137931
16627,2020-05-31,서울 중구,황학동,28761.22,0.134195,0.083456,0.061629,0.070615,0.070454,0.120260,...,0.000000,0.510067,0.020134,0.040268,0.000000,0.194631,0.000000,0.100671,0.033557,0.100671


In [24]:
table.query('STD_YMD <= "2019-02-28"')['SH_LODGE_20_F']

,STD_YMD,GU_NM,HDONG_NM,POP_CNT,AG_20_F,AG_30_F,AG_40_F,AG_50_F,AG_60_F,AG_20_M,...,CJ_FURN,CJ_BOOK,CJ_H_APPL,CJ_HEALTH,CJ_SPORT,CJ_FOOD,CJ_PREG,CJ_CLOTH,CJ_ACC,CJ_BEAUTY
0,2019-02-01,대구 수성구,고산1동,133361.08,0.094684,0.066612,0.117062,0.099566,0.070804,0.104180,...,0.006006,0.342342,0.042042,0.093093,0.012012,0.108108,0.003003,0.195195,0.045045,0.153153
1,2019-02-01,대구 수성구,고산2동,254912.57,0.077246,0.067818,0.099493,0.087825,0.068341,0.096442,...,0.018182,0.331818,0.081818,0.081818,0.004545,0.059091,0.000000,0.168182,0.100000,0.154545
2,2019-02-01,대구 수성구,고산3동,57764.08,0.091888,0.073853,0.112215,0.098319,0.071726,0.098732,...,0.040230,0.264368,0.060345,0.137931,0.008621,0.089080,0.005747,0.178161,0.074713,0.140805
3,2019-02-01,대구 수성구,두산동,125647.42,0.077910,0.078633,0.113528,0.091201,0.078017,0.082605,...,0.007299,0.328467,0.051095,0.116788,0.000000,0.043796,0.000000,0.211679,0.102190,0.138686
4,2019-02-01,대구 수성구,만촌1동,132277.61,0.079484,0.067034,0.095607,0.091221,0.075020,0.098307,...,0.010753,0.301075,0.064516,0.112903,0.000000,0.075269,0.005376,0.252688,0.064516,0.112903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,2019-02-28,서울 중구,중림동,55982.38,0.097276,0.094078,0.076756,0.064753,0.055374,0.075748,...,0.032389,0.107962,0.083671,0.203779,0.009447,0.188934,0.005398,0.156545,0.090418,0.121457
1928,2019-02-28,서울 중구,청구동,35702.61,0.109902,0.098254,0.083244,0.064143,0.070169,0.086854,...,0.041940,0.091743,0.094364,0.193971,0.018349,0.153342,0.006553,0.222805,0.068152,0.108781
1929,2019-02-28,서울 중구,필동,160332.33,0.116170,0.086140,0.078299,0.066108,0.073820,0.085980,...,0.029366,0.165379,0.119011,0.182380,0.013910,0.156105,0.004637,0.146832,0.075734,0.106646
1930,2019-02-28,서울 중구,황학동,66792.63,0.111768,0.091593,0.072710,0.063277,0.059751,0.090595,...,0.060577,0.054808,0.073077,0.185577,0.008654,0.155769,0.009615,0.250962,0.084615,0.116346
